In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import pandas as pd
import calendar
import matplotlib.pyplot as plt
import seaborn as sns
import squarify
import gc
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import time
import datetime
from datetime import datetime
import calendar

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans

sns.set_style('white')

pd.options.display.float_format = '{:,.2f}'.format

# Cargamos el dataset base:

In [ ]:
df_sorted = pd.read_pickle('/kaggle/input/easymoney/EasyMoney_base.pkl',compression='zip')

# Creamos variable totalCuentas:

Vamos a sacar una variable que sea tieneCuentas donde recogeremos si tiene algun producto de los catalogados como cuentas.

In [ ]:
cuentas_easymoney = ['debit_card','em_account_p','em_account_pp',
                                     'em_acount','emc_account','payroll','payroll_account']

In [ ]:
df_cuentas=df_sorted.melt(id_vars=['pk_partition','pk_cid'],
              value_vars=cuentas_easymoney,
              var_name='Product',
              value_name='Count')

In [ ]:
df_cuentas=df_cuentas.groupby(['pk_partition','pk_cid']).agg({'Count':np.sum}).reset_index(drop=False)

In [ ]:
df_cuentas.rename(columns={'Count':'totalCuentas'}, inplace=True)

In [ ]:
df_cuentas.head()

In [ ]:
len(df_sorted),len(df_cuentas)

In [ ]:
df_sorted=pd.merge(df_sorted,df_cuentas, how="inner",on=['pk_cid','pk_partition'])

In [ ]:
del df_cuentas

In [ ]:
gc.collect()

In [ ]:
df_sorted['totalCuentas'].value_counts()

Vemos que casi todo el mundo tiene una cuenta em_acount. 

# Variable total ahorro/inversión:

Vamos a sacar una variable que sea tieneAhorro donde recogeremos si tiene algun producto de los catalogados como ahorro/inversion.

In [ ]:
ahorro_easymoney = ['funds','long_term_deposit','mortgage','pension_plan',
                                     'securities','short_term_deposit']

In [ ]:
df_ahorro=df_sorted.melt(id_vars=['pk_partition','pk_cid'],
              value_vars=ahorro_easymoney,
              var_name='Product',
              value_name='Count')

In [ ]:
df_ahorro=df_ahorro.groupby(['pk_partition','pk_cid']).agg({'Count':np.sum}).reset_index(drop=False)

In [ ]:
df_ahorro.rename(columns={'Count':'totalAhorro'}, inplace=True)

In [ ]:
df_ahorro

In [ ]:
df_sorted=pd.merge(df_sorted,df_ahorro, how="inner",on=['pk_cid','pk_partition'])

In [ ]:
df_sorted['totalAhorro'].value_counts()

Los productos de ahorro son mas minoritarios

In [ ]:
del df_ahorro

# Variable total financiacion:

Vamos a sacar una variable que sea tieneInversion donde recogeremos si tiene algun producto de los catalogados como inversion.

In [ ]:
financiacion_easymoney= ['loans','credit_card']

In [ ]:
df_financiacion=df_sorted.melt(id_vars=['pk_partition','pk_cid'],
              value_vars=financiacion_easymoney,
              var_name='Product',
              value_name='Count')

In [ ]:
df_financiacion=df_financiacion.groupby(['pk_partition','pk_cid']).agg({'Count':np.sum}).reset_index(drop=False)

In [ ]:
df_financiacion.rename(columns={'Count':'totalFinanciacion'}, inplace=True)

In [ ]:
df_sorted=pd.merge(df_sorted,df_financiacion, how="inner",on=['pk_cid','pk_partition'])

In [ ]:
del df_financiacion

In [ ]:
df_sorted['totalFinanciacion'].value_counts()

Los productos de financiacion son minoritarios.

# Segmentacion

Vamos a segmentar nuestros clientes en funcion de las variables: 'totalAssets','totalCuentas','totalAhorro','totalFinanciacion'

In [ ]:
variable_segmentacion=['totalAssets','totalCuentas','totalAhorro','totalFinanciacion']

Lo hacemos solo para los clientes activos (Con producto Easymoney) el ultimo mes.

In [ ]:
clientes_actuales=df_sorted[(df_sorted['pk_partition']=='2019-05-28') &
          (df_sorted['isActive']==1)]

Solo para los clientes activos(con producto Easymoney) el ultimo mes:

Vamos a ver un poco la informacion que tenemos antes de segmentar:

In [ ]:
clientes_actuales[variable_segmentacion]

In [ ]:
pipe = Pipeline(
        steps=[
            ('StandardScaler', StandardScaler()),
            ('KMeans', KMeans(n_clusters=6))
        ]
)

In [ ]:
pipe.fit(clientes_actuales[variable_segmentacion])

In [ ]:
clientes_actuales['Cluster'] = pipe.predict(clientes_actuales[variable_segmentacion])

In [ ]:
clientes_actuales.head(3)

Vamos a ver un poco los grupos que se han generado:

In [ ]:
clientes_actuales.groupby('Cluster').agg({
                                         'totalAssets':np.mean,
                                         'totalCuentas':np.mean,
                                         'totalAhorro':np.mean,
                                         'totalFinanciacion':np.mean
                                        })

Me gusta:  
* Grupo 0: 2 productos del tipo cuentas. **Clientes solo cuentas plus**.
* Grupo 1: 1 solo producto cuentas. (**Clientes solo cuentas**)
* Grupo 2: 2 productos cuentas y uno ahorro. **Clientes cuentas/ahorro**, 
* Grupo 3: 1 producto financiacion y uno cuentas. **Clientes financiacion/cuentas**.
* Grupo 4: 3 productos cuentas, uno ahorro y uno financiaCION. **lOS MEJORES CLIENTES**
* Grupo 5: 3 productos cuentas, uno ahorro. **Clientes cuentas/ahorro Plus**. 

In [ ]:
# para evitar error: Data must have variance to compute a kernel density estimate.
sns.distributions._has_statsmodels = False
sns.pairplot(clientes_actuales.sample(10000), vars=variable_segmentacion, hue='Cluster', aspect=1.5)
plt.show()

Echamos un vistazo a los clientes de el grupo 4 ,que es el mejor grupo:

In [ ]:
lista_campos=['pk_cid',
              'isNewClient',
              'isActive',
'debit_card',
'em_account_p','em_account_pp',
'em_acount','emc_account','payroll','payroll_account',
'funds',
'long_term_deposit','mortgage','pension_plan',
'securities','short_term_deposit',
'loans','credit_card',
'totalAssets','totalCuentas','totalAhorro','totalFinanciacion',
'Cluster',
'SalaryQtil']
    


In [ ]:
clientes_actuales[clientes_actuales['Cluster']==4][lista_campos].T

In [ ]:
clientes_actuales[clientes_actuales['Cluster']==4]['SalaryQtil'].hist()